In [1]:
import numpy as np
import pandas as pd

from matrix_operations import norm_matrix
from sklearn.model_selection import train_test_split
from KernelRegression import KernelRegression
from sklearn.kernel_ridge import KernelRidge

from sklearn.metrics import accuracy_score as acc

In [2]:
# Function to transform numerical to classes 

def pred_to_class(pred):
    pred[pred < 0.5] = 0
    pred[pred >= 0.5] = 1
    return pred

In [3]:
# Function that computes the average squared distance between data points
def compute_sigma_heuristic(X_train):
    distance_matrix = norm_matrix(X_train, X_train)
    sigma = 0
    k = 0
    for i in range(X_train.shape[0]):
        for j in range(i, X_train.shape[0]):
            k +=1
            sigma += distance_matrix[i,j]
        
        
    sigma = sigma/(k)    
    sigma = np.sqrt(sigma)

    mu = np.array([sigma])
    
    return mu

# K = 0

## Importing all the data

### Raw data

In [4]:
home_dir = "D:/Documents/University/MVA/Kernel Methods/mva-KMML-challenge/data/"
k = "0"
X_raw_train = pd.read_csv(home_dir+ "Xtr" + k + ".csv")
X_raw_train

,Id,seq
0,0,TCCTGTGCACATCTGCACCCCTGTTGTGGCCACAAAATGATCCGGC...
1,1,TTAAGTGTATATCTAATAATTTTTTTGCCTACATTCCTGTGTTACC...
2,2,GTGCTCAATTAGTTGCCTACAAATAGTAGCCTGGCACAGTGTAAGC...
3,3,CACCTGGAAAATACAAACAGGCGCAAGAAGAGTTAACCCACAGATC...
4,4,AAATCACTGCCTATCCTTGGGCCAAAAGGTTTCTACAGGAAGCTGC...
...,...,...
1995,1995,ACGCTTCGGCACGGCTGGCACGGGCTCCCTGAGGGGCTGTTGGAGA...
1996,1996,CCTGGCCCTTGGCTCTGCCCTGAAGACAACTCACTCAGGTGCATCC...
1997,1997,TCTTCTCTCTGTGTCCTCATGTGCCCTTTGCTCTGTGCACAGGCAT...
1998,1998,GGCACGGTGGCTCATGCCTGTAATCTTAGCACTTTGGGAGGCCAAG...


In [5]:
test_pd = pd.read_csv(home_dir+ "Xte" + k + ".csv")
test_pd

,Id,seq
0,0,AAGGCCGAGCCCGGCGCGGACGCAGGCGGCTCCGGGCGGGCTCAGC...
1,1,TCTGGGCTCTTAATGTAAAGGTTGCCACTGATGCTGTGTCACCAGC...
2,2,GCCCGCACCGCTGGGCTATTTTTAGCGTCCACTAAACTTAGCCGAC...
3,3,GAGGCGCTGGCAATGGACTAGGAAGCTCGGCTGCCGCTGCTACTGC...
4,4,GGCGAGACTCCATCTCTACAGAAAATTTTTTTAAAAATTAGCTGGA...
...,...,...
995,995,GGCATGGCAGGCCAATCAGAGTCTTCCCTGGGACCTTCTCTGGCAG...
996,996,CTTGTTCTGCAAAGGGAGGATATCTTGAAGCAGGGGCTAACAGGTC...
997,997,CCAGACGGAAGTGCATCATTTTGGCCACAGTGTCCCAGGTCGAAAC...
998,998,GAGGGAGTGGTCCTGGGGGGTCTCGAGGGCCATGGGGGGCACTGGA...


### Training

In [6]:
X_pd = pd.read_csv(home_dir+ "Xtr" + k +"_mat100.csv", header=None, delim_whitespace=True)
X_pd

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.010870,0.010870,0.043478,0.010870,0.021739,0.021739,0.000000,0.010870,0.010870,0.010870,...,0.000000,0.032609,0.01087,0.010870,0.010870,0.000000,0.000000,0.010870,0.000000,0.01087
1,0.000000,0.000000,0.010870,0.010870,0.000000,0.010870,0.010870,0.000000,0.000000,0.000000,...,0.010870,0.010870,0.01087,0.021739,0.000000,0.010870,0.021739,0.032609,0.000000,0.00000
2,0.021739,0.010870,0.021739,0.000000,0.000000,0.000000,0.010870,0.010870,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.010870,0.000000,0.021739,0.000000,0.021739,0.021739,0.01087
3,0.021739,0.010870,0.043478,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.010870,...,0.021739,0.000000,0.00000,0.000000,0.000000,0.000000,0.010870,0.000000,0.000000,0.01087
4,0.021739,0.010870,0.010870,0.010870,0.010870,0.010870,0.032609,0.000000,0.010870,0.043478,...,0.010870,0.010870,0.00000,0.000000,0.010870,0.010870,0.032609,0.010870,0.021739,0.01087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000000,0.000000,0.000000,0.010870,0.000000,0.010870,0.000000,0.021739,0.010870,0.032609,...,0.000000,0.010870,0.01087,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.00000
1996,0.010870,0.021739,0.000000,0.000000,0.000000,0.021739,0.010870,0.010870,0.021739,0.000000,...,0.021739,0.010870,0.00000,0.010870,0.000000,0.000000,0.000000,0.000000,0.010870,0.01087
1997,0.010870,0.000000,0.000000,0.000000,0.000000,0.010870,0.010870,0.010870,0.021739,0.000000,...,0.010870,0.010870,0.00000,0.000000,0.021739,0.000000,0.032609,0.032609,0.000000,0.00000
1998,0.010870,0.010870,0.000000,0.000000,0.021739,0.000000,0.010870,0.021739,0.000000,0.032609,...,0.032609,0.000000,0.00000,0.010870,0.000000,0.010870,0.000000,0.000000,0.000000,0.01087


In [7]:
X = X_pd.to_numpy()
X.shape

(2000, 100)

In [8]:
Y_pd = pd.read_csv(home_dir+ "Ytr" + k + ".csv")
Y_pd.head(10)

,Id,Bound
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
5,5,0
6,6,0
7,7,0
8,8,0
9,9,1


In [9]:
Y = Y_pd["Bound"].to_numpy()
Y.shape

(2000,)

In [10]:
Y[:10]

array([0, 1, 1, 1, 1, 0, 0, 0, 0, 1], dtype=int64)

### Test

In [11]:
X_test_pd = pd.read_csv(home_dir+ "Xte" +k + "_mat100.csv", header=None, delim_whitespace=True)
X_test_pd

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,0.021739,0.000000,0.010870,0.021739,0.010870,0.010870,0.021739,0.010870,0.000000,...,0.000000,0.043478,0.000000,0.000000,0.010870,0.000000,0.000000,0.000000,0.032609,0.000000
1,0.000000,0.021739,0.000000,0.021739,0.021739,0.010870,0.010870,0.000000,0.010870,0.010870,...,0.021739,0.021739,0.000000,0.010870,0.010870,0.000000,0.010870,0.010870,0.032609,0.000000
2,0.000000,0.000000,0.010870,0.000000,0.000000,0.021739,0.000000,0.000000,0.010870,0.010870,...,0.010870,0.010870,0.010870,0.010870,0.010870,0.000000,0.021739,0.032609,0.000000,0.000000
3,0.021739,0.000000,0.000000,0.010870,0.010870,0.010870,0.000000,0.000000,0.021739,0.032609,...,0.010870,0.010870,0.000000,0.010870,0.043478,0.000000,0.021739,0.000000,0.000000,0.010870
4,0.010870,0.000000,0.010870,0.021739,0.010870,0.000000,0.000000,0.010870,0.000000,0.032609,...,0.000000,0.000000,0.000000,0.010870,0.010870,0.010870,0.010870,0.000000,0.010870,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000000,0.010870,0.043478,0.010870,0.021739,0.021739,0.021739,0.010870,0.000000,0.010870,...,0.000000,0.021739,0.010870,0.000000,0.010870,0.010870,0.010870,0.000000,0.010870,0.010870
996,0.000000,0.010870,0.000000,0.000000,0.021739,0.010870,0.043478,0.010870,0.000000,0.000000,...,0.010870,0.000000,0.000000,0.021739,0.000000,0.021739,0.010870,0.021739,0.000000,0.000000
997,0.000000,0.021739,0.032609,0.021739,0.000000,0.000000,0.021739,0.000000,0.010870,0.000000,...,0.021739,0.000000,0.010870,0.000000,0.010870,0.010870,0.000000,0.010870,0.010870,0.000000
998,0.010870,0.010870,0.000000,0.010870,0.021739,0.000000,0.010870,0.010870,0.010870,0.000000,...,0.000000,0.021739,0.010870,0.000000,0.000000,0.010870,0.010870,0.000000,0.010870,0.010870


In [12]:
X_test = X_test_pd.to_numpy()
print(X_test.shape)
X_test[0]

(1000, 100)


array([0.        , 0.02173913, 0.        , 0.01086957, 0.02173913,
       0.01086957, 0.01086957, 0.02173913, 0.01086957, 0.        ,
       0.        , 0.        , 0.        , 0.01086957, 0.        ,
       0.02173913, 0.        , 0.        , 0.01086957, 0.0326087 ,
       0.0326087 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.0326087 , 0.0326087 ,
       0.        , 0.02173913, 0.        , 0.01086957, 0.01086957,
       0.02173913, 0.        , 0.        , 0.05434783, 0.        ,
       0.01086957, 0.        , 0.        , 0.02173913, 0.        ,
       0.        , 0.06521739, 0.02173913, 0.        , 0.        ,
       0.02173913, 0.        , 0.        , 0.01086957, 0.02173913,
       0.02173913, 0.01086957, 0.02173913, 0.01086957, 0.01086957,
       0.        , 0.02173913, 0.        , 0.01086957, 0.01086957,
       0.01086957, 0.01086957, 0.01086957, 0.0326087 , 0.        ,
       0.02173913, 0.01086957, 0.01086957, 0.        , 0.     

## Testing the soundness on a train and validation set

In [13]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size =0.2, shuffle = True, random_state = 0)
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

(1600, 100) (1600,) (400, 100) (400,)


In [14]:
# As a heuristic, we use the average squared distance between features for our sigma

"""
Computing sigma estimate
"""
mu = compute_sigma_heuristic(X_train)
print(mu)

[0.16879884]


In [15]:
gamma = 1/mu**2
reg = 1e-5

In [16]:
KR = KernelRegression("RBF", mu)
KR.fit(X_train, Y_train,  reg = reg)
pred_val = KR.predict(X_val)

In [17]:
K = KernelRidge(alpha = reg, kernel = "rbf", gamma = gamma)
K.fit(X_train, Y_train)
pred_SK = K.predict(X_val)

In [18]:
# Checking that our implementation gives the same answer
np.testing.assert_almost_equal(pred_SK, pred_val)

In [19]:
pred_val = pred_to_class(pred_val)

In [20]:
acc(pred_val, Y_val)

0.5775

## Final prediction on the test set

In [21]:
mu = compute_sigma_heuristic(X)
print(mu)

KR = KernelRegression("RBF", mu)
KR.fit(X, Y,  reg = reg)
pred_0 = KR.predict(X_test)

pred_0 = pred_to_class(pred_0)
pred_0 = np.vstack((np.arange(pred_0.shape[0]), pred_0)).T

[0.16989778]


# K = 1

## Importing all the data

### Raw data

In [22]:
home_dir = "D:/Documents/University/MVA/Kernel Methods/mva-KMML-challenge/data/"
k_int = 1
k = str(k_int)

X_raw_train = pd.read_csv(home_dir+ "Xtr" + k + ".csv")
X_raw_train

,Id,seq
0,2000,AGAAATGAAAGTCATCCAAATAGGAAGAGAAGGAGTCAAAATATCC...
1,2001,TTCTCTCCAGCACTCTTGTCACCAACTCACATACCATACAGTTTAC...
2,2002,ACAGCTGACAAGCAGAACCCAGGGTCACAAGGATCCCATTCATGTG...
3,2003,CAAACTTTTCTATGTTTAGCCATGTGTTAATTAATAAGAAATGTTC...
4,2004,CTTCGTGGTTGGTCAAGAACGGCAGGAGGTATTTGCCCATCGGTGC...
...,...,...
1995,3995,TCTTCCTCTGGAGTAGGTGGAGTTTATATCCCTGCTCTCCTCCCCT...
1996,3996,AATGAGATGTTATGCATATTTCAGATTTTTTTCATTTACAAAGGAA...
1997,3997,CAGCGCTCCCGCCCACCCTCCGGTCTCGGCGGCTCTCCAGAGCGTC...
1998,3998,CTGTGCGTAGAACAGAGAAAATTAGGAACTGTCCCTTTAATAAAAC...


In [23]:
test_pd = pd.read_csv(home_dir+ "Xte" + k + ".csv")
test_pd

,Id,seq
0,1000,GCCAGAATTTATAATGATTTCATTTTTTTTAAAACTTGATGCTCAA...
1,1001,ATGCCTCTAGACCCTTGCCCTACCCACATGTATTTCCTGCTGATGT...
2,1002,ACACGGTGCACCCAACTGAATTTAAACTCGGGAACAGTGTAGGCAG...
3,1003,TGCTGATAGTGTGAGCCACCATGCCCAGCCCAGATGTTTTTTTGAT...
4,1004,AGAGAAAAGCAGCTGTTTGGTTCACTGTTCTCATTAGCATTTATAT...
...,...,...
995,1995,CATATCCCAATTTGCCCACTAGAGGTAGCCATTATCTTGAAATCTT...
996,1996,CAAGAGCGAGACTCCTTCTCAAGAAAAAAAAAAAACCAACCAGTAG...
997,1997,CCAGGATGGGCTACAGAGCGAGACTCCGTCTCAAAGGAAAACAAAA...
998,1998,TGGAGGATGAACAGGAACAATACAACCCCAGGCTGAGCACAGTGCT...


### Train data

In [24]:
X_pd = pd.read_csv(home_dir+ "Xtr" + k +"_mat100.csv", header=None, delim_whitespace=True)
X_pd

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,0.000000,0.010870,0.010870,0.000000,0.021739,0.032609,0.021739,0.000000,0.021739,...,0.000000,0.021739,0.021739,0.065217,0.000000,0.000000,0.032609,0.043478,0.021739,0.010870
1,0.010870,0.000000,0.000000,0.000000,0.043478,0.010870,0.010870,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.021739,0.021739,0.010870,0.000000,0.000000,0.032609,0.021739,0.010870
2,0.021739,0.000000,0.000000,0.021739,0.000000,0.010870,0.021739,0.010870,0.000000,0.021739,...,0.010870,0.010870,0.000000,0.010870,0.000000,0.000000,0.021739,0.000000,0.021739,0.010870
3,0.000000,0.000000,0.000000,0.021739,0.000000,0.010870,0.000000,0.010870,0.000000,0.000000,...,0.021739,0.000000,0.010870,0.010870,0.000000,0.000000,0.010870,0.010870,0.021739,0.000000
4,0.000000,0.010870,0.000000,0.010870,0.000000,0.000000,0.010870,0.021739,0.000000,0.021739,...,0.000000,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.010870,0.021739,0.010870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.032609,0.032609,0.010870,...,0.000000,0.032609,0.010870,0.010870,0.021739,0.021739,0.000000,0.000000,0.010870,0.021739
1996,0.021739,0.000000,0.000000,0.010870,0.054348,0.010870,0.021739,0.010870,0.000000,0.000000,...,0.010870,0.000000,0.010870,0.021739,0.010870,0.000000,0.000000,0.043478,0.010870,0.010870
1997,0.010870,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000,0.054348,0.043478,...,0.010870,0.010870,0.000000,0.000000,0.010870,0.141304,0.000000,0.021739,0.032609,0.032609
1998,0.021739,0.032609,0.032609,0.010870,0.021739,0.010870,0.010870,0.032609,0.000000,0.032609,...,0.000000,0.032609,0.010870,0.000000,0.000000,0.000000,0.021739,0.021739,0.000000,0.010870


In [25]:
X = X_pd.to_numpy()
X.shape

(2000, 100)

In [26]:
Y_pd = pd.read_csv(home_dir+ "Ytr" + k + ".csv")
Y_pd.head(10)

,Id,Bound
0,2000,0
1,2001,0
2,2002,1
3,2003,1
4,2004,1
5,2005,0
6,2006,0
7,2007,0
8,2008,1
9,2009,1


In [27]:
Y = Y_pd["Bound"].to_numpy()
Y.shape, Y[:10]

((2000,), array([0, 0, 1, 1, 1, 0, 0, 0, 1, 1], dtype=int64))

### Test data

In [28]:
X_test_pd = pd.read_csv(home_dir+ "Xte" +k + "_mat100.csv", header=None, delim_whitespace=True)
X_test_pd

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,0.000000,0.032609,0.010870,0.065217,0.010870,0.000000,0.010870,0.000000,0.00000,...,0.010870,0.000000,0.010870,0.043478,0.010870,0.000000,0.010870,0.000000,0.010870,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.00000,...,0.000000,0.032609,0.021739,0.000000,0.043478,0.000000,0.000000,0.000000,0.021739,0.032609
2,0.000000,0.032609,0.010870,0.000000,0.000000,0.000000,0.010870,0.010870,0.010870,0.01087,...,0.032609,0.010870,0.000000,0.010870,0.021739,0.000000,0.000000,0.000000,0.032609,0.010870
3,0.021739,0.000000,0.021739,0.010870,0.086957,0.000000,0.000000,0.000000,0.021739,0.00000,...,0.010870,0.000000,0.000000,0.021739,0.000000,0.000000,0.010870,0.010870,0.043478,0.000000
4,0.010870,0.032609,0.000000,0.010870,0.010870,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.010870,0.000000,0.010870,0.000000,0.010870,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.021739,0.000000,0.010870,0.000000,0.010870,0.010870,0.021739,0.000000,0.000000,0.01087,...,0.010870,0.000000,0.010870,0.010870,0.010870,0.000000,0.000000,0.021739,0.021739,0.000000
996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.01087,...,0.000000,0.010870,0.010870,0.010870,0.010870,0.000000,0.097826,0.000000,0.000000,0.021739
997,0.010870,0.010870,0.032609,0.043478,0.000000,0.010870,0.000000,0.021739,0.010870,0.01087,...,0.010870,0.010870,0.000000,0.021739,0.010870,0.000000,0.108696,0.010870,0.000000,0.010870
998,0.010870,0.010870,0.021739,0.000000,0.000000,0.021739,0.010870,0.032609,0.010870,0.01087,...,0.000000,0.000000,0.010870,0.010870,0.021739,0.010870,0.032609,0.000000,0.021739,0.000000


In [29]:
X_test = X_test_pd.to_numpy()
print(X_test.shape)

(1000, 100)


## Testing the soundness on a train and validation set

In [30]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size =0.2, shuffle = True, random_state = 0)
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

(1600, 100) (1600,) (400, 100) (400,)


In [31]:
# As a heuristic, we use the average squared distance between features for our sigma

"""
Computing sigma estimate
"""
mu = compute_sigma_heuristic(X_train)
print(mu)

[0.17206958]


In [32]:
gamma = 1/mu**2
reg = 1e-5

In [33]:
KR = KernelRegression("RBF", mu)
KR.fit(X_train, Y_train,  reg = reg)
pred_val = KR.predict(X_val)

In [34]:
K = KernelRidge(alpha = reg, kernel = "rbf", gamma = gamma)
K.fit(X_train, Y_train)
pred_SK = K.predict(X_val)

In [35]:
# Checking that our implementation gives the same answer
np.testing.assert_almost_equal(pred_SK, pred_val)

In [36]:
pred_val = pred_to_class(pred_val)

In [37]:
acc(pred_val, Y_val)

0.5675

## Final prediction on the test set

In [38]:
mu = compute_sigma_heuristic(X)
print(mu)

KR = KernelRegression("RBF", mu)
KR.fit(X, Y,  reg = reg)
pred_1 = KR.predict(X_test)

pred_1 = pred_to_class(pred_1)
pred_1 = np.vstack((np.arange(pred_1.shape[0])+ 1000*k_int, pred_1)).T

[0.17152814]


In [39]:
pred_0, pred_1

(array([[  0.,   1.],
        [  1.,   0.],
        [  2.,   1.],
        ...,
        [997.,   1.],
        [998.,   1.],
        [999.,   0.]]),
 array([[1.000e+03, 0.000e+00],
        [1.001e+03, 1.000e+00],
        [1.002e+03, 1.000e+00],
        ...,
        [1.997e+03, 0.000e+00],
        [1.998e+03, 0.000e+00],
        [1.999e+03, 1.000e+00]]))

# K = 2

## Importing all the data

### Raw data

In [40]:
home_dir = "D:/Documents/University/MVA/Kernel Methods/mva-KMML-challenge/data/"
k_int = 2
k = str(k_int)

X_raw_train = pd.read_csv(home_dir+ "Xtr" + k + ".csv")
X_raw_train

,Id,seq
0,4000,GTGCCGACGCAGCGGTGTTGCACCTCCCTCTCCGGCTCTGCTGCCC...
1,4001,CCTGAGTTTGGTGTTCTGTGGAATTATTGAACCTAAGTAGGTAGTG...
2,4002,CCAGGTGCTTACCATTATCATGTAAGTTCTCTAACTGTTTCATGTG...
3,4003,AGAGCGGTGTGGGGGTCACTCAAGAGAACGTGCTGTACCTTGTCAG...
4,4004,CCCCAGGGGCAGGCGGGCTAGCCCCCCCATCATATGCAAATCATGG...
...,...,...
1995,5995,GCTGCCTCTTGGACATCAGACCAAGGCTTACATTCTGAATCCACAG...
1996,5996,CGGTGGCTTATGCCTGTAATGCCAGCACTTTGGAAGGCCGAGGTGG...
1997,5997,GACAACAGCCAGCAGGAACCACCTGTGCTGGGCCCCCGCAGCGAGG...
1998,5998,ACTTGTATGAGGGCAAGACCAACTTCTTCAAAGCCAGTCACTCAAA...


In [41]:
test_pd = pd.read_csv(home_dir+ "Xte" + k + ".csv")
test_pd

,Id,seq
0,2000,TGAGCAGCTCCTGCGGAACCTCTGGCTGGCACAGAGCCCGGGGGCG...
1,2001,CATGCACCACCATGCCTGGCTAAGTTTTGTGTTTTTAGTAGAGACG...
2,2002,TCTGGTCCGTGCAGGCACAACCCAGGCTCCGGCTGGGGGTGGAGTA...
3,2003,TTAGGGATGAAGTGAAAATTTCAAAAGCCCCCTTTTTCAGGGTCTG...
4,2004,TGCAGATGCAAATGCAGGAAACTCCAGGCCATGTGTGGACTGAATG...
...,...,...
995,2995,CCCGCAAGCTGAGGGAGCCGGCTCTGGCCTTGGCCAGTCCAGAAAG...
996,2996,GGGTTTCCCTGTGTTGCCCAGGCTAGTCTTAAATTCCTGGGCTCAA...
997,2997,AGTGGTGCAATCTTGGCTCACTGCAACATCCGCCTCCCAGGTTCAA...
998,2998,CAGGTAGATCACCTGAGGTCAGTTCGAGACCAGCCTGGCCAATGTG...


### Train

In [42]:
X_pd = pd.read_csv(home_dir+ "Xtr" + k +"_mat100.csv", header=None, delim_whitespace=True)
X_pd

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.021739,0.010870,0.021739,0.021739,0.032609,0.021739,0.010870,0.010870,0.01087,0.000000,...,0.000000,0.010870,0.010870,0.010870,0.032609,0.000000,0.000000,0.000000,0.021739,0.000000
1,0.010870,0.000000,0.000000,0.000000,0.010870,0.010870,0.010870,0.010870,0.01087,0.043478,...,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.010870,0.000000,0.010870,0.000000
2,0.000000,0.032609,0.032609,0.021739,0.032609,0.010870,0.010870,0.010870,0.00000,0.021739,...,0.010870,0.010870,0.021739,0.021739,0.000000,0.010870,0.010870,0.010870,0.010870,0.010870
3,0.010870,0.021739,0.010870,0.010870,0.021739,0.010870,0.000000,0.000000,0.00000,0.021739,...,0.000000,0.010870,0.021739,0.010870,0.021739,0.010870,0.010870,0.000000,0.021739,0.000000
4,0.000000,0.000000,0.010870,0.010870,0.032609,0.010870,0.021739,0.000000,0.01087,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.032609,0.010870,0.021739,0.010870,0.000000,0.010870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.021739,0.021739,0.010870,0.010870,0.010870,0.010870,0.010870,0.021739,0.00000,0.021739,...,0.000000,0.000000,0.010870,0.010870,0.010870,0.043478,0.000000,0.043478,0.010870,0.010870
1996,0.000000,0.000000,0.000000,0.010870,0.010870,0.021739,0.000000,0.010870,0.01087,0.021739,...,0.021739,0.010870,0.032609,0.021739,0.000000,0.021739,0.010870,0.010870,0.000000,0.032609
1997,0.000000,0.021739,0.000000,0.010870,0.000000,0.000000,0.010870,0.032609,0.01087,0.010870,...,0.000000,0.010870,0.000000,0.000000,0.021739,0.000000,0.000000,0.010870,0.021739,0.000000
1998,0.010870,0.000000,0.021739,0.010870,0.000000,0.010870,0.010870,0.021739,0.00000,0.000000,...,0.000000,0.000000,0.021739,0.010870,0.000000,0.032609,0.000000,0.000000,0.000000,0.000000


In [43]:
X = X_pd.to_numpy()
X.shape

(2000, 100)

In [44]:
Y_pd = pd.read_csv(home_dir+ "Ytr" + k + ".csv")
Y_pd.head(10)

,Id,Bound
0,4000,1
1,4001,1
2,4002,1
3,4003,0
4,4004,1
5,4005,1
6,4006,0
7,4007,0
8,4008,0
9,4009,0


In [45]:
Y = Y_pd["Bound"].to_numpy()
Y.shape, Y[:10]

((2000,), array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0], dtype=int64))

### Test data

In [46]:
X_test_pd = pd.read_csv(home_dir+ "Xte" +k + "_mat100.csv", header=None, delim_whitespace=True)
X_test_pd

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.021739,0.021739,0.000000,0.010870,0.000000,0.021739,0.021739,0.032609,0.010870,0.000000,...,0.000000,0.010870,0.00000,0.021739,0.00000,0.000000,0.021739,0.010870,0.021739,0.000000
1,0.000000,0.010870,0.021739,0.010870,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,...,0.010870,0.010870,0.01087,0.000000,0.01087,0.010870,0.000000,0.021739,0.000000,0.010870
2,0.021739,0.021739,0.010870,0.021739,0.032609,0.010870,0.000000,0.000000,0.010870,0.021739,...,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.010870,0.021739,0.000000
3,0.000000,0.010870,0.021739,0.010870,0.021739,0.000000,0.010870,0.000000,0.010870,0.010870,...,0.010870,0.010870,0.00000,0.000000,0.00000,0.000000,0.010870,0.010870,0.000000,0.010870
4,0.010870,0.010870,0.010870,0.010870,0.021739,0.010870,0.010870,0.010870,0.000000,0.021739,...,0.010870,0.021739,0.00000,0.000000,0.00000,0.010870,0.000000,0.000000,0.010870,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.000000,0.000000,0.000000,0.010870,0.000000,0.021739,0.000000,0.010870,0.010870,0.000000,...,0.000000,0.010870,0.00000,0.000000,0.00000,0.000000,0.010870,0.000000,0.000000,0.021739
996,0.021739,0.010870,0.021739,0.010870,0.021739,0.010870,0.000000,0.010870,0.000000,0.010870,...,0.010870,0.043478,0.01087,0.000000,0.00000,0.000000,0.000000,0.000000,0.010870,0.000000
997,0.010870,0.010870,0.010870,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.021739,0.010870,0.01087,0.000000,0.00000,0.000000,0.000000,0.000000,0.010870,0.000000
998,0.010870,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,...,0.010870,0.010870,0.01087,0.010870,0.00000,0.010870,0.010870,0.000000,0.000000,0.021739


In [47]:
X_test = X_test_pd.to_numpy()
print(X_test.shape)

(1000, 100)


## Testing the soundness on a train and validation set

In [48]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size =0.2, shuffle = True, random_state = 0)
print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

(1600, 100) (1600,) (400, 100) (400,)


In [49]:
# As a heuristic, we use the average squared distance between features for our sigma

"""
Computing sigma estimate
"""
mu = compute_sigma_heuristic(X_train)
print(mu)

[0.16608455]


In [50]:
gamma = 1/mu**2
reg = 1e-5

In [51]:
KR = KernelRegression("RBF", mu)
KR.fit(X_train, Y_train,  reg = reg)
pred_val = KR.predict(X_val)

In [52]:
K = KernelRidge(alpha = reg, kernel = "rbf", gamma = gamma)
K.fit(X_train, Y_train)
pred_SK = K.predict(X_val)

In [53]:
# Checking that our implementation gives the same answer
np.testing.assert_almost_equal(pred_SK, pred_val)

In [54]:
pred_val = pred_to_class(pred_val)

In [55]:
acc(pred_val, Y_val)

0.685

## Final prediction on the test set

In [56]:
mu = compute_sigma_heuristic(X)
print(mu)

KR = KernelRegression("RBF", mu)
KR.fit(X, Y,  reg = reg)
pred_2 = KR.predict(X_test)

pred_2 = pred_to_class(pred_2)
pred_2 = np.vstack((np.arange(pred_2.shape[0])+ 1000*k_int, pred_2)).T

[0.16643195]


In [57]:
pred_0, pred_1, pred_2

(array([[  0.,   1.],
        [  1.,   0.],
        [  2.,   1.],
        ...,
        [997.,   1.],
        [998.,   1.],
        [999.,   0.]]),
 array([[1.000e+03, 0.000e+00],
        [1.001e+03, 1.000e+00],
        [1.002e+03, 1.000e+00],
        ...,
        [1.997e+03, 0.000e+00],
        [1.998e+03, 0.000e+00],
        [1.999e+03, 1.000e+00]]),
 array([[2.000e+03, 1.000e+00],
        [2.001e+03, 0.000e+00],
        [2.002e+03, 0.000e+00],
        ...,
        [2.997e+03, 0.000e+00],
        [2.998e+03, 0.000e+00],
        [2.999e+03, 1.000e+00]]))

In [58]:
np.testing.assert_almost_equal(pred_0, pred_1)

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatched elements: 1535 / 2000 (76.8%)
Max absolute difference: 1000.
Max relative difference: 1.
 x: array([[  0.,   1.],
       [  1.,   0.],
       [  2.,   1.],...
 y: array([[1.000e+03, 0.000e+00],
       [1.001e+03, 1.000e+00],
       [1.002e+03, 1.000e+00],...

In [59]:
np.testing.assert_almost_equal(pred_0, pred_2)

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatched elements: 1496 / 2000 (74.8%)
Max absolute difference: 2000.
Max relative difference: 1.
 x: array([[  0.,   1.],
       [  1.,   0.],
       [  2.,   1.],...
 y: array([[2.000e+03, 1.000e+00],
       [2.001e+03, 0.000e+00],
       [2.002e+03, 0.000e+00],...

In [60]:
np.testing.assert_almost_equal(pred_1, pred_2)

AssertionError: 
Arrays are not almost equal to 7 decimals

Mismatched elements: 1509 / 2000 (75.5%)
Max absolute difference: 1000.
Max relative difference: 1.
 x: array([[1.000e+03, 0.000e+00],
       [1.001e+03, 1.000e+00],
       [1.002e+03, 1.000e+00],...
 y: array([[2.000e+03, 1.000e+00],
       [2.001e+03, 0.000e+00],
       [2.002e+03, 0.000e+00],...

# Concatenating the whole predictions

In [61]:
final_pred = np.vstack((pred_0, pred_1, pred_2))
print(final_pred.shape)

(3000, 2)


In [63]:
pd.DataFrame(final_pred, columns = ["Id", "Bound"]).to_csv("KRR_pred.csv")